In [1]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [4]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [5]:
def make_counts(labels):    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
import hdbscan
seed = 123
np.random.seed(seed)

def shift(l, n):
    return l[n:] + l[:n]

def trackML31(df, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
              z_shift, dz_val):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
#     dz0  = -0.00070
    dz0 = dz_val
    stepdz =  0.00001
    stepeps =  0.000005
    z = z + z_shift
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    x2 = x/r
    y2 = y/r
    
    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    x1 = x/rt
    y1 = y/rt
    
    y3 = np.log1p(abs(y/r))*np.sign(y)
    
    theta = np.arctan2(rt, z)
    theta_deg = np.degrees(np.arctan2(rt, z))
    tt = np.tan(theta_deg)
    
    z4 = np.sqrt(abs(z/rt))
    
    x4 = np.sqrt(abs(x/r))
    y4 = np.sqrt(abs(y/r))
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
#         a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+ 0.0000145*rt**2)/1000*(ii/2)/180*np.pi
    
        da1 = mm*(1 + (2 * 0.0000145 * rt))/1000*(ii/2)/180*np.pi
        da2 = mm * (2 * 0.0000145)/1000*(ii/2)/180*np.pi
        dz = mm*(dz0 + ii*stepdz)
        a1 = a0 + dz*z*np.sign(z)
        aa1 = a1 - 2*da1 + da2
        saa1 = np.sin(aa1)
        caa1 = np.cos(aa1)
        ia1 = a0*rt + mm*(((rt**2)/2) + (0.0000145*rt**3)/3)/1000*(ii/2)/180*np.pi
        
        sa1 = np.sin(da1)
        ca1 = np.cos(da1)
        
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        raa = x*caa + y*saa
        
        t1 = theta+mm*(rt+ 0.8435*rt**2)/1000*(ii/2)/180*np.pi
        ctt = np.cos(t1)
        stt = np.sin(t1)
        ttt = np.tan(t1)
        
        
        r0Inv = 2. * np.cos(a1 - t1) / r
        r0Inv1 = 2. * np.cos(a1 - t1) / rt
        
        t0 = a0 - theta
        b1 = t0+mm*(rt+ 0.0000005*rt**2)/1000*(ii/2)/180*np.pi
        r0Inv2 = 2. * np.cos(b1) / r
        
        deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, z3, ctt, stt]))
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
                                                            ctt, stt, z4, x4, y4, raa, r0Inv, da1, r/ca1, 
                                                            saa1, caa1]))
    
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
#         print(X.shape)
        
#         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w7]
        cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w10, w10, w11, w12, w13, w14, w15, w16, w17, w18, w18]
        cx = [w1,w1,w2,w3]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
#         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,cluster_selection_method='leaf',metric='euclidean', leaf_size=50)
#         l = cl.fit_predict(X)+1
        
        l= DBSCAN(eps=0.004,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         l= DBSCAN(eps=w7,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    lss = []
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
    
    lss.append(labels)
    
#     for i in range(Niter):
#         labels1 = np.zeros(num_hits,np.int32)
#         counts1 = np.zeros(num_hits,np.int32)
#         ls1 = ls.copy()
#         ls1 = shift(ls1, 1)
#         np.random.shuffle(ls1)
#         for l in ls1:
#             c = make_counts(l)
#             idx = np.where((c-counts>0) & (c<20))[0]
#             labels1[idx] = l[idx] + labels1.max()
#             counts1 = make_counts(labels1)
#         l1 = labels1.copy()
#         lss.append(l1)


#     labels = np.zeros(num_hits,np.int32)
#     counts = np.zeros(num_hits,np.int32)
    
#     for l in lss:
#         c = make_counts(l)
#         idx = np.where((c-counts>0) & (c<20))[0]
#         labels[idx] = l[idx] + labels.max()
#         counts = make_counts(labels)
    

#     sub = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
#         data=np.column_stack(([int(0),]*len(df), df.hit_id.values, labels))
#     )
#     sub['track_count'] = sub.groupby('track_id')['track_id'].transform('count')

    
#     sub1 = sub[sub.track_id <  0]
#     sub2 = sub[sub.track_id >= 0]
#     L_neg = sub1.track_id.values
#     L_pos = sub2.track_id.values
#     a = 1
#     for l in L_neg:
#         for l1 in range(a, np.iinfo(np.int32).max):
#             if l1 in L_pos:
#                 continue
#             sub.loc[sub.track_id == l, 'track_id'] = l1
#             a = l1 +1
#             break
    
#     L = list(sub.track_id.values)
#     labels = np.zeros(num_hits,np.int32)
#     for ii in range(num_hits):
#         labels[ii] = L[ii]
#     print(np.any(labels < 0))
    return labels

In [7]:
def Fun4BO2(dz_val):
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.01201
    w6 = 0.0003864
    w7 = 0.0205
    w8 = 0.0049
    w9 = 0.00121
    w10 = 1.4930496676654575e-05
    w11 = 0.0318
    w12 = 0.000435
    w13 = 0.00038
    w14 = 0.00072
    w15 = 0.000265
    
    w16 = 0.0031
    w17 = 0.0001
    w18 = 0
#     Niter=179
#     Niter=246
    Niter=247
    
    print(dz_val)
    z_shift = 5.5
#     dz = z_shift
#     hits.z = hits.z.values + dz
    
#     Niter = Niter.astype(np.int32)
    track_id = trackML31(hits, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
                         5.5, dz_val)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [8]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
#     hits.z = hits.z.values + 5.5
    
    w1 = [0.1, 1.2]
    w2 = [0.1, 1.2]
    w3 = [0.1, 1.2]
#     Niter = [140, 190]
    
    
# #     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
# #     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
#     w1 = 1.1932215111905984
#     w2 = 0.39740553885387364
#     w3 = 0.3512647720585538
#     w4 = [0.1, 0.2] # 0.1470 -> 0.55690
#     w4 = 0.1470
#     w5 = [0.001, 1.2] # 0.7781 -> 0.55646, 0.7235 + N = 247 => 0.56025
    
#     Niter = 179
#     Niter = 247
#     w5 = 0.01
# #     for w6 in [0.012, 0.01201, 0.01202, 0.01203, 0.01204, 0.01205, 0.01206, 0.01207, 0.01208, 0.01209, 0.0121]:
#     EPS = 1e-12
#     w6 = [0.001, 1.2]
#     w6 = 0.0205
#     w8 = [0.00008, 0.0005]
#     w13 = 0.00038
#     for w2 in np.arange(0.01, 3.5, 0.005):
#         print(w2)
#     Fun4BO2(1)
#     Fun4BO2(1.95)
#     for w18 in [0.09, 0.095, 0.099, 0.101]:
#         print(w18)
#         Fun4BO2(w18)
    for dz_val in np.arange(-0.00070, 0.00070, 0.00001):
        print(dz_val)
        Fun4BO2(dz_val)
#     Niter = [240, 480]
#     w3 = [0.1, 1.5]
#     w1 = [1.4, 2.5]
#     bo = BayesianOptimization(Fun4BO2,pbounds = {'w1':w1, 'w2':w2, 'w3':w3})
#     bo.maximize(init_points = 20, n_iter = 5, acq = "ucb", kappa = 2.576)

# x/y: 7 | 06m30s |    0.55302 |    0.0100 | 
# x/y: 0.001: 0.55949
# x/y: 0.0001: 0.55949
# x/y: 0.002: 0.55959
# x/y: 0.003: 0.55915
# x/y: 0.0025: 0.55925
# x/y: 0.0015: 0.55953
# x/r: 0.0015: 0.56186
# x/r: 0.002: 0.56334
# x/r: 0.0025: 0.563989
# x/r: 0.003: 0.56447
# x/r: 0.01: 0.569822
# x/r: 0.015: 0.56940
# x/r: 0.012: 0.5719
# x/r: 0.01201: 0.57192
# 1.4499999999999993e-05 * rt**2: 0.5720702851970194
# 0.0000145
# z3: 10 | 07m12s |    0.57208 |    0.0205 | 
# count: 19: 0.572567, 17: 0.57263
# ctt, stt after change:  2 | 07m56s |    0.57345 |    0.0001 | (0.00010567777727496665)
# x4: 25 | 09m42s |    0.57359 |    0.0002 | (0.000206214286412982)
# x4: 0.000435 (0.5737387485278771) (x4 = np.sqrt(abs(x/r)))
# w13: 00038 (ctt,stt): 0.5737528800479372
# ensemble of 10: 0.5772859116242378
# ensemble of Niter=247 (random shuffle): 0.5775039093976666
# w10: 9e-06: 0.5737885478380893
# w10: 5e-06: 0.5738467299283372
# test z-shifting
# -0.055, eps = 0.0035: 0.07681636873598773
# -5.5, eps = 0.005: 0.546315038619802
# -5.5, eps = 0.0042: 0.5582823239770327
# -5.5, eps = 0.0042, w1 = 1.95, 0.5285485364616167
# -5.5, eps = 0.0035, w1 = 1.95, 0.0.517231947322158
# -5.5 eps = 0.0042, w2 = 0.4850 (24 | 07m10s |    0.52859 |    0.4850 | )
# -5.5 eps = 0.0042, w3 = 0.3131 (23 | 07m35s |    0.52884 |    0.3131 | )
# z-shifting dz0 = -0.00080, stepdz =  0.00001, 0.5209422031729637 (w1,w2,w3 only)
# z-shifting (dz = 0) same as above
# z-shifting (z_shift=0 dz= 0 + ii*step_dz, eps = 0.0032, 0.553517167182983 for w1, w2, w3)
# z-shifting (z_shift=0.0055 dz= 0.0055 + ii*step_dz, eps = 0.0032, 0.5535734115757227 for w1, w2, w3 (asme for 0.0055))
# z_shifting z-shift= 0, 0.553517167182983
# z_shift 20 random: 0.36869323586813313
# org: 0.5860334935502527
# testing r/cos (0.001 - 0.5840812114731859, 0.0001: 0.5860832334979645, 9e-05: 0.5860832334979645)
# testing ia1 (0.001: 0.5839413212088399, 0.01:less, 0.0001: 0.5860832334979645, 1e-05: same )
# testing saa1, caa1 (0.1: 0.5854467527377532)
# chemist z kernel (instead of 0.45 in R kernel got 0.5053554962578144)
# z-shift = -5.5 local, 0.49168517829207886
# z-shift = -5.5, global, 0.4593534891830958
# z-shift = 5.5, global, 0.46466047502820357
# z-shift, 5.5, local, 0.49366025115549256

-0.0007
-0.0007
 179.0000000[ 0] score : 0.48707212
 179.0000000[ 1] score : 0.49217007
 179.0000000[ 2] score : 0.49422078
 179.0000000[ 3] score : 0.49504771
 179.0000000[ 4] score : 0.49495487
 179.0000000[ 5] score : 0.49544610
 179.0000000[ 6] score : 0.49521603
 179.0000000[ 7] score : 0.49515433
--------------------------------------
0.49366025115549256
-0.00069
-0.00069
 179.0000000[ 0] score : 0.48350686
 179.0000000[ 1] score : 0.48944507
 179.0000000[ 2] score : 0.49084851
 179.0000000[ 3] score : 0.49175513
 179.0000000[ 4] score : 0.49241138
 179.0000000[ 5] score : 0.49252134
 179.0000000[ 6] score : 0.49204870
 179.0000000[ 7] score : 0.49181161
--------------------------------------
0.4905435742584656
-0.0006799999999999999
-0.0006799999999999999
 179.0000000[ 0] score : 0.48641038
 179.0000000[ 1] score : 0.49135926
 179.0000000[ 2] score : 0.49319270
 179.0000000[ 3] score : 0.49418362
 179.0000000[ 4] score : 0.49402097
 179.0000000[ 5] score : 0.49455970
 179.000000

 179.0000000[ 0] score : 0.48442484
 179.0000000[ 1] score : 0.48941907
 179.0000000[ 2] score : 0.48969894
 179.0000000[ 3] score : 0.49069246
 179.0000000[ 4] score : 0.49154623
 179.0000000[ 5] score : 0.49144614
 179.0000000[ 6] score : 0.49106926
 179.0000000[ 7] score : 0.49091917
--------------------------------------
0.48990201516073256
-0.0004799999999999994
-0.0004799999999999994
 179.0000000[ 0] score : 0.48795947
 179.0000000[ 1] score : 0.49324567
 179.0000000[ 2] score : 0.49481664
 179.0000000[ 3] score : 0.49643090
 179.0000000[ 4] score : 0.49608799
 179.0000000[ 5] score : 0.49693595
 179.0000000[ 6] score : 0.49627277
 179.0000000[ 7] score : 0.49672667
--------------------------------------
0.4948095079703885
-0.0004699999999999994
-0.0004699999999999994
 179.0000000[ 0] score : 0.48436370
 179.0000000[ 1] score : 0.48937141
 179.0000000[ 2] score : 0.49018552
 179.0000000[ 3] score : 0.49114717
 179.0000000[ 4] score : 0.49203912
 179.0000000[ 5] score : 0.49190258

 179.0000000[ 0] score : 0.48485713
 179.0000000[ 1] score : 0.49060111
 179.0000000[ 2] score : 0.49189807
 179.0000000[ 3] score : 0.49294171
 179.0000000[ 4] score : 0.49352427
 179.0000000[ 5] score : 0.49353425
 179.0000000[ 6] score : 0.49332052
 179.0000000[ 7] score : 0.49351076
--------------------------------------
0.49177347595495496
-0.00026999999999999887
-0.00026999999999999887
 179.0000000[ 0] score : 0.48290615
 179.0000000[ 1] score : 0.48829560
 179.0000000[ 2] score : 0.48949785
 179.0000000[ 3] score : 0.49063655
 179.0000000[ 4] score : 0.49101807
 179.0000000[ 5] score : 0.49162362
 179.0000000[ 6] score : 0.49103331
 179.0000000[ 7] score : 0.49092208
--------------------------------------
0.4894916527589844
-0.00025999999999999884
-0.00025999999999999884
 179.0000000[ 0] score : 0.48435317
 179.0000000[ 1] score : 0.49012294
 179.0000000[ 2] score : 0.49148429
 179.0000000[ 3] score : 0.49224110
 179.0000000[ 4] score : 0.49285783
 179.0000000[ 5] score : 0.4929

 179.0000000[ 0] score : 0.46955003
 179.0000000[ 1] score : 0.47420558
 179.0000000[ 2] score : 0.47575398
 179.0000000[ 3] score : 0.47645751
 179.0000000[ 4] score : 0.47701999
 179.0000000[ 5] score : 0.47735665
 179.0000000[ 6] score : 0.47713363
 179.0000000[ 7] score : 0.47712812
--------------------------------------
0.4755756864572024
-5.9999999999998314e-05
-5.9999999999998314e-05
 179.0000000[ 0] score : 0.46173109
 179.0000000[ 1] score : 0.46718620
 179.0000000[ 2] score : 0.46950378
 179.0000000[ 3] score : 0.46992401
 179.0000000[ 4] score : 0.47069412
 179.0000000[ 5] score : 0.47149333
 179.0000000[ 6] score : 0.47085804
 179.0000000[ 7] score : 0.47158826
--------------------------------------
0.46912235507596123
-4.999999999999829e-05
-4.999999999999829e-05
 179.0000000[ 0] score : 0.46454089
 179.0000000[ 1] score : 0.46926698
 179.0000000[ 2] score : 0.47107830
 179.0000000[ 3] score : 0.47168440
 179.0000000[ 4] score : 0.47224883
 179.0000000[ 5] score : 0.472841

KeyboardInterrupt: 